## Парсинг HTML (hh.ru, superjob.ru)

#### Библиотеки для парсинга HTML и данных

In [171]:
import pandas as pd
import requests
import re
import json

from bs4 import BeautifulSoup as bs
from pprint import pprint

## Скачивание данных

#### 1. Подготовка параметров для скачивания

In [114]:
vacancy_name = input('Введите название вакансии для поиска: ')

Введите название вакансии для поиска: Директор по маркетингу


In [115]:
pages = int(input('Введите количество страниц для поиска: '))

Введите количество страниц для поиска: 1


In [116]:
vacancy_name, pages

('Директор по маркетингу', 1)

In [117]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
           'Accept': '*/*'}

In [194]:
hh_link = 'https://hh.ru'
sj_link = 'https://superjob.ru'

vacancies = []

for page in range(pages):
    #Поиск по странице hh.ru
    hh_params = {'page':str(page),
                 'text':vacancy_name,
                 'L_save_area':'false',
                 'clusters':'true',
                 'search_field':'name',
                 'enable_snippets':'true'}
    response = requests.get(hh_link+'/search/vacancy', headers=headers, params=hh_params)
    soup = bs(response.text, 'html.parser')
    
    if response.ok:
        vacancies_block = soup.find('div',{'class':'vacancy-serp'})
        vacancies_list = vacancies_block.find_all('div',{'class':'vacancy-serp-item'})
        
        print(len(vacancies_list))
        
        for vacancy in vacancies_list:
            vacancy_data = {}
            #id вакансии
            vacancy_data['name'] = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'}).getText()
            vacancy_data['link'] = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'})['href']
            a = vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'})
            if a == None:
                a = None
            else:
                a = a.getText()
                a = re.split('[\xa0 -]',a)
                a = [i for i in a if i != '—']
                
                if len(a) == 4 and a[0] == 'от':
                    vacancy_data['min_salary'] = int(a[1]+a[2])
                if len(a) == 4 and a[0] == 'до':
                    vacancy_data['max_salary'] = int(a[1]+a[2])
                if len(a) > 4:
                    vacancy_data['min_salary'] = int(a[0]+a[1])
                    vacancy_data['max_salary'] = int(a[2]+a[3])
                    
                vacancy_data['currency'] = a[-1]
                    
            b = vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-address'}).getText()
            vacancy_data['city'] = b.split(',')[0]
            vacancy_data['main_link'] = hh_link
            vacancy_id = vacancy.find('script',{'data-name':'HH/VacancyResponsePopup/VacancyResponsePopup'})
            vacancy_data['vacancy_id'] = json.loads(vacancy_id['data-params'])['vacancyId']
            
            vacancies.append(vacancy_data)
            
    #Поиск по странице superjob.ru
    
    sj_params = {'page':str(page),
                 'keywords':vacancy_name,
                 'noGeo':1}
    
    response = requests.get(sj_link+'/vacancy/search/', headers=headers, params=sj_params)
    soup = bs(response.text, 'html.parser')
    
    if response.ok:
        vacancies_block = soup.find('div',{'class':'iJCa5'})
        vacancies_list = vacancies_block.find_all('div',{'class':'LvoDO'})
        
        print(len(vacancies_list))
        
        for vacancy in vacancies_list:
            vacancy_data = {}
            vacancy_data['name'] = vacancy.find('a',{'class':'icMQ_'}).getText()
            vacancy_data['link'] = sj_link+vacancy.find('a',{'class':'icMQ_'})['href']
            
            a = vacancy.find('span',{'class':'PlM3e'}).getText()
            if a =='По договорённости':
                a = None
            else:
                a = re.split('[\xa0 -]',a)
                a = [i for i in a if i != '—']
                
                if len(a) == 4 and a[0] == 'от':
                    vacancy_data['min_salary'] = int(a[1]+a[2])
                if len(a) == 4 and a[0] == 'до':
                    vacancy_data['max_salary'] = int(a[1]+a[2])
                if len(a) > 4:
                    vacancy_data['min_salary'] = int(a[0]+a[1])
                    vacancy_data['max_salary'] = int(a[2]+a[3])
                    
                vacancy_data['currency'] = a[-1]
                    
            b = vacancy.find('span',{'class':'clLH5'}).find_next_sibling().getText()
            vacancy_data['city'] = b.split(',')[0]
            vacancy_data['main_link'] = sj_link
            vacancy_data['vacancy_id'] = str(''.join(re.findall('[0-9]+', vacancy_data['link'])))
            
            vacancies.append(vacancy_data)

50
20


In [195]:
vacancies_data = pd.DataFrame(vacancies)
vacancies_data

,name,link,min_salary,max_salary,currency,city,main_link,vacancy_id
0,Директор по маркетингу и развитию,https://novorossiysk.hh.ru/vacancy/38195470?qu...,150000.0,250000.0,руб.,Москва,https://hh.ru,38195470
1,Директор по маркетингу,https://novorossiysk.hh.ru/vacancy/38123801?qu...,100000.0,300000.0,руб.,Москва,https://hh.ru,38123801
2,Директор по маркетингу,https://novorossiysk.hh.ru/vacancy/38148877?qu...,230000.0,NaN,руб.,Санкт-Петербург,https://hh.ru,38148877
3,Директор по маркетингу и развитию обменного пу...,https://novorossiysk.hh.ru/vacancy/37415755?qu...,250000.0,500000.0,руб.,Санкт-Петербург,https://hh.ru,37415755
4,Директор по маркетингу и рекламе,https://novorossiysk.hh.ru/vacancy/36331693?qu...,150000.0,250000.0,руб.,Москва,https://hh.ru,36331693
...,...,...,...,...,...,...,...,...
65,Руководитель отдела маркетинга и коммуникаций,https://superjob.ru/vakansii/rukovoditel-otdel...,NaN,NaN,NaN,Москва,https://superjob.ru,33645453
66,Руководитель отдела трейд-маркетинга,https://superjob.ru/vakansii/rukovoditel-otdel...,NaN,NaN,NaN,Санкт-Петербург,https://superjob.ru,34098535
67,Коммерческий директор (строительство),https://superjob.ru/vakansii/kommercheskij-dir...,NaN,NaN,NaN,Москва,https://superjob.ru,34181990
68,Коммерческий директор,https://superjob.ru/vakansii/kommercheskij-dir...,180000.0,NaN,руб.,Санкт-Петербург,https://superjob.ru,33697122


# Mongo DB

1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

In [19]:
from pymongo import MongoClient

In [196]:
client = MongoClient('127.0.0.1',27017)
db = client['vacancies_db']

vacancies_db = db.vacancies

In [197]:
vacancies_db.insert_many(vacancies)

2. Функция, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)

In [198]:
x = int(input('Введите заработную плату для поиска: '))

Введите заработную плату для поиска: 150000


In [205]:
for vacancy in vacancies_db.find({'$or':[
                                      {'min_salary': {'$gt': x}}, {'max_salary': {'$gt': x}}
                                  ]},
                                {'_id':0, 'name': 1, 'min_salary': 1, 'max_salary': 1}):
    print(vacancy)

{'name': 'Директор по маркетингу и развитию', 'min_salary': 150000, 'max_salary': 250000}
{'name': 'Директор по маркетингу', 'min_salary': 100000, 'max_salary': 300000}
{'name': 'Директор по маркетингу', 'min_salary': 230000}
{'name': 'Директор по маркетингу и развитию обменного пункта криптовалют', 'min_salary': 250000, 'max_salary': 500000}
{'name': 'Директор по маркетингу и рекламе', 'min_salary': 150000, 'max_salary': 250000}
{'name': 'Директор по маркетингу и рекламе', 'min_salary': 150000, 'max_salary': 250000}
{'name': 'Директор по маркетингу e-commerce', 'min_salary': 200000}
{'name': 'Директор по маркетингу', 'min_salary': 80000, 'max_salary': 200000}
{'name': 'Директор по маркетингу', 'min_salary': 170000}
{'name': 'Директор по маркетингу и рекламе', 'min_salary': 150000, 'max_salary': 300000}
{'name': 'Директор по маркетингу и рекламе', 'min_salary': 150000, 'max_salary': 300000}
{'name': 'Директор по маркетингу / Онлайн-Маркетолог', 'min_salary': 50000, 'max_salary': 300000

3. Функция, которая будет добавлять в базу данных только новые вакансии с сайта

In [209]:
for vacancy in vacancies:
    vac_id = vacancy['vacancy_id']
    vacancies_db.update_one(
        {'vacancy_id': vac_id},
        {'$set': vacancy},
        upsert=True)

In [210]:
vacancies_db.count()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


70